# Postagens do INSTAGRAM

## Segunda parte

A segunda parte da entrega consiste em um novo relatório sobre os posts das marcas. 

### Etapa 1 - Buscar & Salvar os dados no BQ

Para cada Post (por conta):

- Comentários
  - Número de comentários em cada postagem
  
- Likes
  - Número de likes em cada postagem
  
- Compartilhamentos
  - Número de compartilhamentos

- Tipo de post
  - Pode ser Reel, Carrossel, Post etc

- Link da imagem do post

- Descrição/legenda
  - Registrar o que a conta escreveu como legenda

- Contas marcadas
  - Registrar quais foram as contas marcadas na postagem

- Hashtags usadas
  - Registrar quais foram as hashtags adicionadas na postagem

- Data de publicação
  - Dia que o post foi publicado

- Data da última busca/atualização
  - Dia que buscamos os dados


### Etapa 2 - Tratar os dados do BQ e gerar relatório

Para o relatório (por conta):
- Média de comentários
  - Qtde de comentários nos posts da semana / Qtde posts na semana
- Média de likes
  - Qtde de Likes nos posts da semana / Qtde posts na semana
- Média de compartilhamentos
  - Qtde de Compartilhamentos nos posts da semana / Qtde posts na semana
- Quantidade de posts por semana


# Buscamos dados das seguintes marcas:

## Próprias: @
- mmartanoficial (MMartan)
- santistadecora (Santista)
- artex (Artex)

## Concorrentes: @
- artelasse (Artelasse)
- camicado (Camicado)
- casaalmeidaoficial (Casa Almeida)
- casariachuelo (Casa Riachuelo)
- karstenoficial (Karsten)
- mundodoenxoval (Mundo do Enxoval)
- trussardioficial (Trussardi)
- zeloloja (Zelo)

## Lista dos usuários


@ - "altenburg.oficial",

@ - "altenburghaus",

@ - "artelasse",

@ - "artex",

@ - "buddemeyeroficial",

@ - "casaalmeidaoficial",

@ - "casariachuelo",

@ - "casa.sonno",

@ - "hoomybr",

@ - "karstenoficial",

@ - "mmartanoficial",

@ - "santistadecora",

@ - "trussardioficial",

@ - "trousseauoficial",

@ - "zeloloja"


In [3]:
list_of_tracked_users = [
    "altenburg.oficial",
    "altenburghaus",
    "artelasse",
    "artex",
    "buddemeyeroficial",
    "casaalmeidaoficial",
    "casariachuelo",
    "casa.sonno",
    "hoomybr",
    "karstenoficial",
    "mmartanoficial",
    "santistadecora",
    "trussardioficial",
    "trousseauoficial",
    "zeloloja"
]

## Formatando URL e parametros para a requisição

In [22]:
from commom.instagram_data.instagram_data_formater import InstagramDataFormater


data_formater = InstagramDataFormater()

for username in list_of_tracked_users:
    service_params = data_formater.get_service_params("instagram_scrapper_api", username)
    headers = data_formater.format_params_to_headers("instagram_scrapper_api", service_params)
    posts_url = data_formater.format_media_url("instagram_scrapper_api", service_params)
    media_querystring = data_formater.format_media_querystring("instagram_scrapper_api", service_params)


## Tratamento da resposta

- Mock da resposta
- Criação do objeto
- Criação do dataframe inicial
- Merge de várias respostas em um dataframe

### Mock


In [47]:
from commom.instagram_data.instagram_data_formater import InstagramDataFormater
from typing import Dict, List
import json

def load_local_json(filename: str) -> Dict[str, str]:
    file = 'posts_responses/'+filename
    with open(file, 'r', encoding='utf-8') as f:
        return json.load(f)


local_username_files = [
    'artex.json',
    'artex_2.json',
    'casariachuelo.json',
    'casariachuelo_2.json',
    'karstenoficial.json',
    'karstenoficial_2.json',
    'mmartanoficial.json',
    'mmartanoficial_2.json',
    'trussardioficial.json',
    'trussardioficial_2.json'
]

def mock_get(username : str):
    data_formater = InstagramDataFormater()
    
    service_params = data_formater.get_service_params("instagram_scrapper_api", username)
    headers = data_formater.format_params_to_headers("instagram_scrapper_api", service_params)
    posts_url = data_formater.format_media_url("instagram_scrapper_api", service_params)
    media_querystring = data_formater.format_media_querystring("instagram_scrapper_api", service_params)
    
    if username in local_username_files:
        return load_local_json(username)
    else:
        return load_local_json(local_username_files[0])

## DataClass -> Post

In [48]:
import dataclasses
from dataclasses import dataclass
import datetime
from typing import Any, List, Optional, Union
import pandas as pd

@dataclass
class Post:
    code: str
    comment_count: int
    is_pinned: bool
    is_video: bool
    like_and_view_counts_disabled: bool
    like_count: int
    media_name: str
    share_count: int
    user_id: Union[int, str]
    username: str
    created_at_utc: Union[int, str, Any]
    id: Union[int, str]
    last_update: Union[datetime.datetime,datetime.date, str]
    text: Optional[str] = ''
    ig_play_count: Optional[int] = 0
    is_paid_partnership: Optional[bool] = False
    play_count: Optional[int] = 0
    did_report_as_spam: Optional[bool] = False
    hashtags: Optional[Any] = None
    mentions: Optional[Any] = None
    video_duration: Optional[Any] = None
    carousel_media_count: Optional[int] = None
    carousel_media_ids: Optional[Any] = None
    
    def asdict(self):
        return dataclasses.asdict(self)
    
    def to_dataframe(self) -> pd.DataFrame:
        return pd.DataFrame(data=self.asdict(), columns=self.asdict().keys(), index=[0])
    
    def keys():
        return Post.__dataclass_fields__.keys()
    

## api JSON response -> Post Obj

In [49]:
from typing import List, Optional, Union
import pandas as pd

def parse_post_response(jsonObj) -> List[Post]:
    data = jsonObj['data']
    items = data["items"]
    count = data["count"]
    
    postsList : List[Post] = []
    
    if count > 0:
        for item in items:
            caption = item['caption']
            user = item["user"]
            carousel_media_count = 0
            carousel_media_ids = '[]'
            is_paid_partnership = False
            ig_play_count = 0
            play_count = 0
            share_count = 0
            video_duration = 0 
            is_pinned = False           
            
            if 'carousel_media_count' in item.keys():
                carousel_media_count = item['carousel_media_count']
                carousel_media_ids = str(item['carousel_media_ids'])
            
            if 'is_paid_partnership' in item.keys():
                is_paid_partnership = item['is_paid_partnership']
            
            if 'ig_play_count' in item.keys():
                ig_play_count = item['ig_play_count']
                play_count = item['play_count']
            
            if 'share_count' in item.keys():
                share_count = item['share_count']
            
            if 'video_duration' in item.keys():
                video_duration = item['video_duration']
            
            if 'is_pinned' in item.keys():
                is_pinned = item['is_pinned']
                
            post = Post(
                code= item['code'],
                comment_count= item['comment_count'],
                ig_play_count= ig_play_count,
                play_count= play_count,
                is_paid_partnership = is_paid_partnership,
                is_pinned= is_pinned,
                is_video= item['is_video'],
                like_and_view_counts_disabled= item['like_and_view_counts_disabled'],
                like_count= item['like_count'],
                media_name= item['media_name'],
                share_count= share_count,
                user_id= user['id'],
                username= user['username'],
                created_at_utc= caption['created_at_utc'],
                id= caption['id'],
                text= caption['text'],
                did_report_as_spam= caption['did_report_as_spam'],
                hashtags= str(caption['hashtags']),
                mentions= str(caption['mentions']),
                video_duration= video_duration,
                carousel_media_count= carousel_media_count,
                carousel_media_ids= carousel_media_ids,
                last_update= pd.to_datetime('today', format="%Y-%m-%d", utc=True),
            )
            postsList.append(post)
    
    return postsList

In [26]:
response = mock_get('artex.json')
parsedResponse = parse_post_response(response)
parsedResponse

[Post(code='DBPi75WA7cx', comment_count=13, is_pinned=False, is_video=True, like_and_view_counts_disabled=False, like_count=310, media_name='reel', share_count=10, user_id='13503206', username='juliamunhao', created_at_utc=1729206041, id='3481154686583813937', last_update=Timestamp('2024-11-18 10:27:26.143928+0000', tz='UTC'), text='Melhorar a qualidade do meu sono melhorou tudo por aqui… e esses são os meus rituais pra ter uma boa noite de sono, com @artex 🌙\n\nProdutos da linha de Básicos que eu mais amo:\n- Pillow Top Matelassê Bálsamo\n- Jogo de lençol cetim 210 fios 100% algodão\n- Edredom Matelassê Habitual\n\n*parceria', ig_play_count=11275, is_paid_partnership=False, play_count=11275, did_report_as_spam=False, hashtags='[]', mentions="['@artex']", video_duration=56.966, carousel_media_count=0, carousel_media_ids='[]'),
 Post(code='DBPC338vdLs', comment_count=0, is_pinned=False, is_video=False, like_and_view_counts_disabled=False, like_count=155, media_name='album', share_count=

## Post List -> to dataframe

In [27]:
import pandas as pd

response = mock_get('artex.json')
parsedResponse = parse_post_response(response)
df = pd.DataFrame(columns=Post.keys())
for post in parsedResponse:
    if len(df) > 0 :
        df = df.reset_index(drop=True)
    df = pd.concat([df, post.to_dataframe()])

/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_26770/3792251352.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, post.to_dataframe()])


In [28]:
local_username_files = [
    'artex.json',
    'artex_2.json',
    'casariachuelo.json',
    'casariachuelo_2.json',
    'karstenoficial.json',
    'karstenoficial_2.json',
    'mmartanoficial.json',
    'mmartanoficial_2.json',
    'trussardioficial.json',
    'trussardioficial_2.json'
]

def flux0(usernamesList):
    df = pd.DataFrame(columns=Post.keys())
    for username in usernamesList:
        response = mock_get(username)
        parsedResponse = parse_post_response(response)
        for post in parsedResponse:
            if len(df) > 0 :
                df = df.reset_index(drop=True)
             
            df = pd.concat([df, post.to_dataframe()])
    
    return df

complete_df = flux0(local_username_files)
complete_df

/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_26770/3672741046.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, post.to_dataframe()])


,code,comment_count,is_pinned,is_video,like_and_view_counts_disabled,like_count,media_name,share_count,user_id,username,...,text,ig_play_count,is_paid_partnership,play_count,did_report_as_spam,hashtags,mentions,video_duration,carousel_media_count,carousel_media_ids
0,DBPi75WA7cx,13,False,True,False,310,reel,10,13503206,juliamunhao,...,Melhorar a qualidade do meu sono melhorou tudo...,11275,False,11275,False,[],['@artex'],56.966,0,[]
1,DBPC338vdLs,0,False,False,False,155,album,0,2077874975,artex,...,"O que é o que é? Super macia, traz aconchego p...",0,False,0,False,[],[],0.000,3,"[3481013661697241371, 3481013661546310769, 348..."
2,DBNGTK5PTZK,14,False,False,False,442,album,0,2077874975,artex,...,Cadê o time estampados? Com a delicada do jogo...,0,False,0,False,[],[],0.000,3,"[3480465784159683396, 3480465784377937735, 348..."
3,DBKMN8LpTdF,2,False,False,False,132,album,0,2077874975,artex,...,Denúncia: fizemos uma enquete no canal de tran...,0,False,0,False,[],[],0.000,4,"[3479647387996632118, 3479647387921054317, 347..."
4,DBFepaeo1vg,4,False,True,False,86,reel,1,2077874975,artex,...,"Se você não passou por isso, com certeza foi a...",5496,False,5921,False,[],[],15.015,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,DAi7g7Rxwxh,2,False,False,False,72,post,0,595865967,trussardioficial,...,La Dolce Vita italiana.\n\nMomentos que captur...,0,False,0,False,[],[],0.000,0,[]
116,DAgiSPOxO_V,57,False,True,False,772,reel,38,7838811,helena_lunardelli,...,Dream bed! 💭🤍 Nova coleção @trussardioficial -...,31227,False,31227,False,[],['@trussardioficial'],27.566,0,[]
117,DAgCf2ix1LM,0,False,True,False,111,reel,26,595865967,trussardioficial,...,Limoncino captura a essencia vibrante de Cinqu...,555122,False,555122,False,[],[],25.058,0,[]
118,DAdxBzwB7cU,2,False,False,False,145,album,0,595865967,trussardioficial,...,A Festa dos Limões em Cinque Terre é um espetá...,0,False,0,False,"['#DolceRiviera', '#CinqueTerre', '#Trussardi']",[],0.000,7,"[3467142901434674129, 3467142901434678079, 346..."


## Fluxo 1

-  Dados serão montados para o formato da API
-  A resposta será um JSON
-  O JSON será formatado para Objeto Post -> List [ Post ]
-  Os objetos serão montados num dataframe
   -  Vamos subir os primeiros dados para o BQ
   -  Encerrar primeiro fluxo parte 0
-  - 
-  Devemos concatenar com um dataframe anterior (que virá do BQ)
-  Devemos subir novamente o dataframe concatenado (para o BQ)

### DATA MANAGER

In [ ]:
from typing import Any, List
import pandas
from commom.instagram_data.instagram_data_formater import InstagramDataFormater
import requests
import time

def fetch_user_media_info(instagram_account_list: List[str], debugMode: bool = True) -> Any:
    # logger.info("Fetching user info - init")
    current_service = "instagram_scrapper_api"
    # current_service = self.services_availables[0]
    posts_list: List[Post] = []
    # logger.info(f"Service: {current_service}")
    
    for username in instagram_account_list:
        service_params = InstagramDataFormater().get_service_params(current_service, username)
        headers = InstagramDataFormater().format_params_to_headers(current_service, service_params)
        posts_url = InstagramDataFormater().format_media_url(current_service, service_params)
        media_querystring = InstagramDataFormater().format_media_querystring(current_service, service_params)
        
        try:
            res = requests.get(url=posts_url,headers=headers, params=media_querystring)
            response = res.json()
            
            parsedResponse = parse_post_response(response)
            
            for post in parsedResponse:
                posts_list.append(post)

            time.sleep(1)
        except Exception as e:
            print(e)
    
    return posts_list

def _create_empty_posts_dataset() -> pd.DataFrame:
    df = pd.DataFrame(columns=Post.keys())
    return df

def update_posts_dataset(dataset: pd.DataFrame, posts_list: List[Post]) -> pd.DataFrame:
    dataset['last_update'] = pd.to_datetime(dataset['last_update'], format="%Y-%m-%d", utc=True)
    df_temp = _create_empty_posts_dataset()
    
    for post in posts_list:
        df_temp = pd.concat([df_temp, post.to_dataframe()])
    
    df_temp = df_temp.reset_index(drop=True)    
    
    dataset = pd.concat([dataset, df_temp])
    
    dataset = dataset.reset_index(drop=True)
    
    return dataset


### Monitor SERVICE

In [110]:
from commom.database.data_handler import DataHandler
from commom.database.queries.query_instagram_posts import QUERY_INSTAGRAM_POSTS

data_handler = DataHandler()


def update_posts(listOfUsernames, debugMode: bool = True):
    # Get post info
    postsList = fetch_user_media_info(listOfUsernames, debugMode)
    
    if debugMode:
        df_history : pd.DataFrame = pd.read_pickle('instagram_posts.pkl')
        
        datasetNew = update_posts_dataset(df_history, postsList)
        
        data_handler.write_to_pickle(datasetNew, 'instagram_posts.pkl')
    
    else:
        df_history = data_handler.read_from_bigquery(query=QUERY_INSTAGRAM_POSTS)
        
        datasetNew = update_posts_dataset(df_history, postsList)
        
        datasetNew = data_handler.write_to_bigquery(dataset_id='innovation_dataset', table_id='instagram_posts', dataframe=datasetNew)
        
    return datasetNew

In [ ]:
usernamesList = [
    "altenburg.oficial",
    "altenburghaus",
    "artelasse",
    "artex",
    "buddemeyeroficial",
    "casaalmeidaoficial",
    "casariachuelo",
    "casa.sonno",
    "hoomybr",
    "karstenoficial",
    "mmartanoficial",
    "santistadecora",
    "trussardioficial",
    "trousseauoficial",
    "zeloloja"
]

df = update_posts(usernamesList, False)


[Post(code='DCfAukvy8Hz', comment_count=5, is_pinned=False, is_video=False, like_and_view_counts_disabled=False, like_count=58, media_name='album', share_count=0, user_id='1256753216', username='altenburg.oficial', created_at_utc=1731872419, id='3503522235670053363', last_update=Timestamp('2024-11-18 15:07:15.291042+0000', tz='UTC'), text='Selo Farofa de aprovação! \n\nO Jogo de Cama Maré em 300 fios de Cetim em puro algodão é escolha perfeita para relaxar neste domingo, né? \n\nO cetim de 300 fios garante um toque macio e envolvente, ideal para criar um ambiente de descanso perfeito! Que tal reproduzir essa cama posta aí na sua casa?', ig_play_count=0, is_paid_partnership=False, play_count=0, did_report_as_spam=False, hashtags='[]', mentions='[]', video_duration=0, carousel_media_count=2, carousel_media_ids='[3503522226971023227, 3503522227189110195]'), Post(code='DCcG_pbuQ0A', comment_count=0, is_pinned=False, is_video=True, like_and_view_counts_disabled=False, like_count=45, media_n

/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_26770/1866185866.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_temp = pd.concat([df_temp, post.to_dataframe()])


,code,comment_count,is_pinned,is_video,like_and_view_counts_disabled,like_count,media_name,share_count,user_id,username,...,text,ig_play_count,is_paid_partnership,play_count,did_report_as_spam,hashtags,mentions,video_duration,carousel_media_count,carousel_media_ids
0,DCejzeuOxu3,3,False,False,False,126,post,0,2077874975,artex,...,✋🏻🚨 Pare agora de rolar o feed! Você acaba de ...,0,False,0,False,[],['@acasadanath'],0,0,[]
1,DCcI-enO1c5,1,False,True,False,67,reel,0,2077874975,artex,...,O encanto do básico está na simplicidade que t...,4389,False,4399,False,[],[],35.827,0,[]
2,DCaQy-5MhfV,2,False,True,False,80,reel,0,2077874975,artex,...,"Arrumar a cama é mais do que organização, é au...",4261,False,4261,False,[],[],46.033,0,[]
3,DCXZlqsSe0m,8,False,True,False,347,reel,9,2077874975,artex,...,Conforto que abraça e cores que encantam! 🌈💫 N...,7392,False,7393,False,[],[],14.615,0,[]
4,DCPzfe7PhV_,2,False,False,False,77,album,0,2077874975,artex,...,"A vida solteira também importa, tá? 😘 Mas você...",0,False,0,False,[],[],0,4,"[3499241886294384755, 3499241886118167797, 349..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,DB_40qRJNmV,3,False,False,False,139,album,0,325716886,zeloloja,...,"Este ano, a Black Zelo já chega com uma novida...",0,False,0,False,"['#Zelo', '#BemEstar', '#Conforto', '#BlackFri...",[],0,2,"[3494761734661878529, 3494761734720622503]"
200,DB9ROkWpYWr,4,False,True,False,95,reel,2,325716886,zeloloja,...,O Novembro Black Zelo chegou! ✨\n\nEssa é a op...,3963,False,3963,False,"['#Zelo', '#Conforto', '#BemEstar', '#BlackFri...",[],26.033,0,[]
201,DBwiEUDpFVE,2,False,False,False,151,album,0,325716886,zeloloja,...,Uma cama arrumada com Zelo Select é a certeza ...,0,False,0,False,"['#Zelo', '#ZeloPorVocê', '#Conforto', '#BemEs...",[],0,6,"[3490439531476637342, 3490439531443224463, 349..."
202,DBrmx5ayo8o,5,False,False,False,72,album,0,325716886,zeloloja,...,Quer uma dica para renovar o visual do seu qua...,0,False,0,False,"['#Zelo', '#ZeloPorVocê', '#Conforto', '#BemEs...",[],0,4,"[3489052883788934974, 3489052883528898845, 348..."


[INNOVATION-LOG] [write_to_bigquery] [Line (41)] [INFO]: Connection to BigQuery successful
[INNOVATION-LOG] [write_to_bigquery] [Line (51)] [INFO]: write done


,code,commentCount,isPinned,isVideo,likeAndViewCountsDisabled,likeCount,mediaName,shareCount,userId,username,...,text,igPlayCount,isPaidPartnership,playCount,didReportAsSpam,hashtags,mentions,videoDuration,carouselMediaCount,carouselMediaIds
18,DB_bPu5tAww,0,False,False,False,23,post,0,42856773759,altenburghaus,...,"No ambiente Rumah Deca, criado pela arquiteto ...",0,False,0,False,"['#AltenburgHaus', '#CasaCor']",['@liosimas'],0,0,[]
28,DCPzfe7PhV_,2,False,False,False,77,album,0,2077874975,artex,...,"A vida solteira também importa, tá? 😘 Mas você...",0,False,0,False,[],[],0,4,"[3499241886294384755, 3499241886118167797, 349..."
93,DCg041cRfIq,1,False,False,False,28,post,0,1502987277,karstenoficial,...,Você acorda e arruma a cama ou já sai correndo...,0,False,0,False,[],[],0,0,[]
